<a href="https://colab.research.google.com/github/KeisukeShimokawa/papers-challenge/blob/master/src/cv/SAM/notebooks/SAM_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip list | grep torch

torch                         1.6.0+cu101    
torchsummary                  1.5.1          
torchtext                     0.3.1          
torchvision                   0.7.0+cu101    


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

まずは確認のために、以下の特徴マップをSAMの入力値と考える。

In [3]:
inputs = torch.randn(10, 64, 16, 16)

print("input feature map: ", inputs.shape)

input feature map:  torch.Size([10, 64, 16, 16])


以下ではパッチワイズのSelf-Attentionを考えていく。

$$\boldsymbol{y}_{i}=\sum_{j \in \mathcal{R}(i)} \alpha\left(\boldsymbol{x}_{R(i)}\right)_{j} \odot \beta\left(\boldsymbol{x}_{j}\right)$$

このパッチワイズの計算のうち、左側のパッチ領域に対する計算では、アダマール積を計算する対象と形状を合わせる$\gamma(\cdot)$と関係性を計算する$\delta(\cdot)$に分解する。

$$\gamma\left(\delta\left(\boldsymbol{x}_{\mathcal{R}(i)}\right)\right)$$

In [4]:
in_planes  = inputs.shape[1] # 入力チャンネル
rel_planes = in_planes // 16 # α
out_planes = in_planes // 4  # β
print(in_planes, rel_planes, out_planes)

# 計算の設定を行う
share_planes = 8
kernel_size = 7
stride = 1
dilation = 1

64 4 16


以下の計算では、Attention計算に利用するパッチ領域に対して、計算コストを削減するため1x1畳み込み演算を使用してチャンネル数を小さくする。

In [5]:
# step1
conv1 = nn.Conv2d(in_planes, rel_planes, kernel_size=1)
conv2 = nn.Conv2d(in_planes, rel_planes, kernel_size=1)
conv3 = nn.Conv2d(in_planes, out_planes, kernel_size=1)

x1, x2, x3 = conv1(inputs), conv2(inputs), conv3(inputs)

print(f"x1: {x1.shape}")
print(f"x2: {x2.shape}")
print(f"x3: {x3.shape}")

x1: torch.Size([10, 4, 16, 16])
x2: torch.Size([10, 4, 16, 16])
x3: torch.Size([10, 16, 16, 16])


Self-Attention計算では、あるピクセル位置での特徴ベクトルと、その周辺のピクセルを含む特徴ベクトルに対してAttention計算を実施する。

そこでまずはQueryとなる各ピクセル位置での特徴ベクトルを計算するために形状を変更する。

具体的には、高さと幅の次元を1つにまとめる。

In [6]:
x1_reshape = x1.view(x1.shape[0], -1, 1, x1.shape[2] * x1.shape[3])

print(f"x1 reshape: {x1_reshape.shape}")

x1 reshape: torch.Size([10, 4, 1, 256])


pytorchのUnfold関数の出力値

$$
(N, C \times \Pi(\text { kernel_size }), L)
$$

この$L$とは以下の計算に従う。なお$d$は空間方向の次元数である。

$$
L=\prod_{d}\left[\frac{\text{spatial_size}[d]+2 \times \text{padding}[d]-\text{dilation}[d] \times\left(\text{kernel_size}[d]-1\right)-1}{\text{stride}[d]}+1\right]
$$

In [7]:
# paddingしたサイズと、Kernel_size-1のサイズが等しくなるので
# 単純に入力された空間次元数をかけ合わせたものになる。(16x16=256)
(((16+3*2) + 2 * 0 - (kernel_size - 1) - 1) + 1) ** 2

256

In [8]:
# C x Product(kernel_size)
4   * kernel_size ** 2

196

ここで行うPaddingとUnfoldのイメージは以下のようなものになる。

![](https://www.dropbox.com/s/ieal9ylytzdga44/SAMPatch.jpg?dl=1)

このイメージに従い、パッチ領域に合わせて特徴マップを変形させていく。

In [9]:
# 設定していたとおり、PaddingとUnfoldを初期化する
unfold_x2 = nn.Unfold(kernel_size=kernel_size, dilation=dilation, padding=0, stride=stride)
pad = nn.ReflectionPad2d(kernel_size // 2)

# カーネルサイズに合わせてPaddingを設定する。
# Paddingは通常の畳み込み演算を行う場合と考え方は変わらない
x2_padded = pad(x2)
print(f"x2 padded: {x2_padded.shape}")

# これで上のイメージ図の右側のパッチ領域を計算できる
x2_unfold = unfold_x2(x2_padded)
print(f"x2 unfold: {x2_unfold.shape}")

# ピクセルごとの特徴ベクトルを計算したテンソルと形状を合わせる
x2_reshape = x2_unfold.view(x1_reshape.shape[0], -1, 1, x1_reshape.shape[-1])
print(f"s2 reshape: {x2_reshape.shape}")

x2 padded: torch.Size([10, 4, 22, 22])
x2 unfold: torch.Size([10, 196, 256])
s2 reshape: torch.Size([10, 196, 1, 256])


これで以下のように特徴ベクトルとパッチ領域のベクトルを取得できた。

$$x_{1}:\left[B, C, 1, H\times{W}\right]$$

$$x_{2}:\left[B, C\times{\text{kernel_size}^2}, 1, H\times{W}\right]$$



In [10]:
print(f"x1 reshape: {x1_reshape.shape}")
print(f"x2 reshape: {x2_reshape.shape}")

x1 reshape: torch.Size([10, 4, 1, 256])
x2 reshape: torch.Size([10, 196, 1, 256])


次に、QueryとKeyの特徴マップを結合させる。

$$\text { Concatenation: } \delta\left(\mathbf{x}_{\mathcal{R}(i)}\right)=\left[\varphi\left(\mathbf{x}_{i}\right),\left[\psi\left(\mathbf{x}_{j}\right)\right]_{V j \in \mathcal{R}(i)}\right]$$


In [11]:
w_concat = torch.cat((x1_reshape, x2_reshape), dim=1)
print(f"w concat: {w_concat.shape}")

w concat: torch.Size([10, 200, 1, 256])


次に結合させた特徴マップを、Valueとなる特徴マップと計算する前に、非線形な変換を行う。

この際には論文中の以下の表に則り、2層のマッピング関数を使用する。

![](https://www.dropbox.com/s/8zway0blqmeopc5/SAM%20mapping.png?dl=1)

以下で計算した特徴マップを$\beta$側に組み込むが、カーネルを共有する設定を設けているため、そのまま出力するのではなく、共有するカーネル分に合わせてチャンネル数を調整する。

In [12]:
# 以下では1x1畳み込みを使用する。その際にはボトルネック構造を採用してチャンネル数を削減している。
conv_w = nn.Sequential(
    nn.BatchNorm2d(rel_planes * (1 + pow(kernel_size, 2))),
    nn.ReLU(inplace=True),
    nn.Conv2d(rel_planes * (1 + pow(kernel_size, 2)), out_planes // share_planes, kernel_size=1, bias=False),
    nn.BatchNorm2d(out_planes // share_planes),
    nn.ReLU(inplace=True),
    nn.Conv2d(out_planes // share_planes, pow(kernel_size, 2) * out_planes // share_planes, kernel_size=1)
)

w_conv = conv_w(w_concat)
print(f"w conv: {w_conv.shape}")

w conv: torch.Size([10, 98, 1, 256])


次にカーネルサイズの大きさの次元を、他の次元に分離させ、各ピクセル位置でのパッチサイズに合わせた、チャンネル数に従う次元をもつ特徴マップを取得することができる。

In [13]:
w_reshape = w_conv.view(x1_reshape.shape[0], -1, pow(kernel_size, 2), x1_reshape.shape[-1])
print(f"w reshape: {w_reshape.shape}")

w reshape: torch.Size([10, 2, 49, 256])


ここまでで$\alpha$部分と$\beta$部分を計算することができた。

ここで`shared_kernel`に従ってAttentionの重みを共有する。

$$\beta\left(\boldsymbol{x}_{j}\right):\left[B, C_{out}, H, W\right]$$

$$\alpha\left(\boldsymbol{x}_{R(i)}\right)_{j}:\left[B, \frac{C_{out}}{\text{groups}}, \text{kernel_size}^2, H\times{W}\right]$$

In [14]:
print('beta feature map:  {}'.format(x3.shape))
print('alpha feature map: {}'.format(w_reshape.shape))

beta feature map:  torch.Size([10, 16, 16, 16])
alpha feature map: torch.Size([10, 2, 49, 256])


In [15]:
unfold_x3 = nn.Unfold(kernel_size=kernel_size, dilation=dilation, padding=0, stride=stride)
pad_x3 = nn.ReflectionPad2d(kernel_size // 2)

x3_padded = pad_x3(x3)
print(x3_padded.shape)

x3_unfold = unfold_x3(x3_padded)
print(x3_unfold.shape)

x3_reshape = x3_unfold.view(x3_padded.shape[0], -1, pow(kernel_size, 2), x3_unfold.shape[-1])
print(x3_reshape.shape)

torch.Size([10, 16, 22, 22])
torch.Size([10, 784, 256])
torch.Size([10, 16, 49, 256])


## Autual Sample

なおカーネルサイズはストライドなど、高さと幅方向に同じ値を設定する場合は、以下のようにスカラー値からタプル形式のオブジェクトを取得できる。



In [ ]:
from torch.nn.modules.utils import _pair


pad_mode = 1


kernel_size_pair = _pair(kernel_size)
stride_pair = _pair(stride)
padding_pair = _pair(kernel_size // 2)
dilation_pair = _pair(dilation)

print(kernel_size_pair)
print(stride_pair)
print(padding_pair)
print(dilation_pair)
print(pad_mode)

In [ ]:
import torch
import torch.nn as nn


class SAM(nn.Module):
    def __init__(self, sa_type, in_planes, rel_planes, out_planes, share_planes, kernel_size=3, stride=1, dilation=1):
        super(SAM, self).__init__()
        self.sa_type, self.kernel_size, self.stride = sa_type, kernel_size, stride
        self.conv1 = nn.Conv2d(in_planes, rel_planes, kernel_size=1)
        self.conv2 = nn.Conv2d(in_planes, rel_planes, kernel_size=1)
        self.conv3 = nn.Conv2d(in_planes, out_planes, kernel_size=1)
        self.conv_w = nn.Sequential(nn.BatchNorm2d(rel_planes * (pow(kernel_size, 2) + 1)),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(rel_planes * (pow(kernel_size, 2) + 1), out_planes // share_planes, kernel_size=1, bias=False),
                                    nn.BatchNorm2d(out_planes // share_planes),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(out_planes // share_planes, pow(kernel_size, 2) * out_planes // share_planes, kernel_size=1))
        self.unfold_i = nn.Unfold(kernel_size=1, dilation=dilation, padding=0, stride=stride)
        self.unfold_j = nn.Unfold(kernel_size=kernel_size, dilation=dilation, padding=0, stride=stride)
        self.pad = nn.ReflectionPad2d(kernel_size // 2)
        # self.aggregation = Aggregation(kernel_size, stride, (dilation * (kernel_size - 1) + 1) // 2, dilation, pad_mode=1)

    def forward(self, x):
        x1, x2, x3 = self.conv1(x), self.conv2(x), self.conv3(x)
        if self.sa_type == 0:  # pairwise
            p = self.conv_p(position(x.shape[2], x.shape[3], x.is_cuda))
            w = self.softmax(self.conv_w(torch.cat([self.subtraction2(x1, x2), self.subtraction(p).repeat(x.shape[0], 1, 1, 1)], 1)))
        else:  # patchwise
            if self.stride != 1:
                x1 = self.unfold_i(x1)
            x1 = x1.view(x.shape[0], -1, 1, x.shape[2]*x.shape[3])
            x2 = self.unfold_j(self.pad(x2)).view(x.shape[0], -1, 1, x1.shape[-1])
            w = self.conv_w(torch.cat([x1, x2], 1)).view(x.shape[0], -1, pow(self.kernel_size, 2), x1.shape[-1])
        print(x3.shape)
        print(w.shape)
        return x

In [ ]:
sam_sample = SAM(sa_type=1,
                 in_planes=in_planes,
                 rel_planes=rel_planes,
                 out_planes=out_planes,
                 share_planes=share_planes,
                 kernel_size=kernel_size,
                 stride=stride,
                 dilation=dilation)

sam_sample

In [ ]:
_ = sam_sample(inputs)

In [ ]:
nn.Conv2d(10, 20, kernel_size=3, stride=1, padding=1, groups=1).weight.shape

In [ ]:
nn.Conv2d(10, 20, kernel_size=3, stride=1, padding=1, groups=2).weight.shape

In [ ]:
nn.Conv2d(4, 6, kernel_size=3, stride=1, padding=1, groups=2).weight.shape